In [1]:
import time

from bokeh.io import vplot, hplot, gridplot
from bokeh.plotting import Session, cursession, output_server, show, figure

from data.datastream import queriesinput_to_syncgens_yf

from candlestick.candlestick import Candlestick
from table.table_deltas import TableDeltas
from scatterplot.scatterplot_deltas import ScatterplotDeltas

In [2]:
session = Session(load_from_config=False, root_url = "http://127.0.0.1:5006/")
output_server("layout", url = "http://127.0.0.1:5006/")
#output_notebook()#Inizializzazione variabili query

#parametri utilizzati per effettuare la query a Yahoo Finance
x_quote_id = "UCG.MI"
y_quote_id = "ISP.MI"
start_y = 2013
start_m = 1
start_d = 1
stop_y = 2015
stop_m = 12
stop_d = 31
step = "d" 

#parametri relativi al grafico CandleStick
tmdict = {"d":1, "w":7, "m":(365 * 4 + 1) / (12 * 4)}
candle_visualized_n = 15
quadwidth_percentual = 0.2
tmstep = tmdict[step] #per i mesi preso step pari ai giorni in 4 anni diviso i mesi in 4 anni

Using saved session configuration for http://127.0.0.1:5006/
To override, pass 'load_from_config=False' to Session


In [3]:
#parametri relativi al widget Table
row_n = 7

s0 = ScatterplotDeltas()

c1 = Candlestick(candle_visualized_n, quadwidth_percentual, tmstep)
t1 = TableDeltas(row_n)

c2 = Candlestick(candle_visualized_n, quadwidth_percentual, tmstep)
t2 = TableDeltas(row_n)

objrow1_list = [s0]
objrow2_list = [c1, t1]
objrow3_list = [c2, t2]

In [4]:
table_and_candlestick_height = 230
candlestick_width = 900
scatter_height = 320
posizione_toolbar = None

#QUESTA PARTE E' DA ELIMINARE PORTANDOLA ALL'INTERNO DELLE RISPETTIVE CLASSI. IN PARTICOLARE
#DECIDERE UN POSIZIIONAMENTO PREDEFINITO DELLE TOOLBAR
#DECIDERE UNA DIMENSIONE PREDEFINITA DEGLI OGGETTI IN BASE A DIMENSIONI RELATIVE E IMPAGINAMENTO
s0.get_plot().toolbar_location = posizione_toolbar
s0.get_plot().xaxis.axis_label = x_quote_id
s0.get_plot().yaxis.axis_label = y_quote_id
s0.get_plot().plot_height = scatter_height
s0.get_plot().plot_width = scatter_height

c1.get_plot().toolbar_location = posizione_toolbar
c1.get_plot().yaxis.axis_label = x_quote_id
c1.get_plot().plot_height = table_and_candlestick_height
c1.get_plot().plot_width = candlestick_width

t1_height = table_and_candlestick_height

c2.get_plot().toolbar_location = posizione_toolbar
c2.get_plot().yaxis.axis_label = y_quote_id
c2.get_plot().plot_height = table_and_candlestick_height
c2.get_plot().plot_width = candlestick_width

t2.height = table_and_candlestick_height
plot = vplot(gridplot([[None, s0.get_plot(), None]]),
          hplot(c1.get_plot(), t1.get_plot()), 
          hplot(c2.get_plot(), t2.get_plot()))

show(plot)

In [5]:
csvlinesgen = queriesinput_to_syncgens_yf((x_quote_id, y_quote_id), start_y, start_m, start_d, stop_y, stop_m, stop_d, step, headers = False)
while True:
    csvlines = next(csvlinesgen)
    to_update = []
    for obj in objrow1_list:
        data = obj.csvlines_to_data_yf(csvlines)
        to_update += obj.update(data)
    for obj in objrow2_list: 
        data = obj.csvlines_to_data_yf([csvlines[0]])
        to_update += obj.update(data)
    for obj in objrow3_list: 
        data = obj.csvlines_to_data_yf([csvlines[1]])
        to_update += obj.update(data)
    cursession().store_objects(*to_update)
    time.sleep(0.5)

StopIteration: 